In [1]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import itertools
import requests
import tempfile
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data = pd.DataFrame(columns=['image_path','label'])
labels = {
    r"C:\Users\rathi\Downloads\cloudy-20250214T050317Z-001\cloudy":"Cloudy",
    r"C:\Users\rathi\Downloads\green_area-20250214T050324Z-001\green_area":"Green_Area",
    r"C:\Users\rathi\Downloads\water-20250214T050330Z-001\water":"Water",
    r"C:\Users\rathi\Downloads\desert-20250214T050321Z-001\desert":"Desert"
}

for folder in labels:
    if not os.path.exists(folder):
        print(f"Warning:The folder {folder} does not exist..")
        continue
    for image_name in os.listdir(folder):
        image_path = os.path.join(folder,image_name)
        if os.path.isfile(image_path):
            label =labels[folder]
            data = pd.concat([data, pd.DataFrame({'image_path':[image_path],'label':[label]})],ignore_index = True)

print(data)

                                             image_path   label
0     C:\Users\rathi\Downloads\cloudy-20250214T05031...  Cloudy
1     C:\Users\rathi\Downloads\cloudy-20250214T05031...  Cloudy
2     C:\Users\rathi\Downloads\cloudy-20250214T05031...  Cloudy
3     C:\Users\rathi\Downloads\cloudy-20250214T05031...  Cloudy
4     C:\Users\rathi\Downloads\cloudy-20250214T05031...  Cloudy
...                                                 ...     ...
2012  C:\Users\rathi\Downloads\desert-20250214T05032...  Desert
2013  C:\Users\rathi\Downloads\desert-20250214T05032...  Desert
2014  C:\Users\rathi\Downloads\desert-20250214T05032...  Desert
2015  C:\Users\rathi\Downloads\desert-20250214T05032...  Desert
2016  C:\Users\rathi\Downloads\desert-20250214T05032...  Desert

[2017 rows x 2 columns]


In [3]:
data.to_csv('image_dataset.csv',index=False)

In [4]:
df = pd.read_csv("image_dataset.csv")

In [5]:
train_df,test_df = train_test_split(df,test_size=0.2,random_state=42)

In [6]:
train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range =0.2,
                                   zoom_range =0.2,
                                   horizontal_flip =True,
                                   rotation_range=45,
                                   vertical_flip =True,
                                  fill_mode ='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_dataframe(dataframe = train_df,
                                                    x_col ="image_path",
                                                    y_col="label",
                                                    target_size=(255,255),
                                                    batch_size=32,
                                                    class_mode="categorical")

test_generator = test_datagen.flow_from_dataframe(dataframe = test_df,
                                                    x_col ="image_path",
                                                    y_col="label",
                                                    target_size=(255,255),
                                                    batch_size=32,
                                                    class_mode="categorical")

Found 1613 validated image filenames belonging to 4 classes.
Found 404 validated image filenames belonging to 4 classes.


In [13]:
model = Sequential()
model.add(Conv2D(32,(3, 3),input_shape=(255,255,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3, 3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

C:\Users\rathi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 253, 253, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,492 (56.61 MB)

 Trainable params: 14,839,492 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_generator,epochs=25,validation_data=test_generator)

C:\Users\rathi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
 2/51 ━━━━━━━━━━━━━━━━━━━━ 1:25 2s/step - accuracy: 0.2109 - loss: 3.6417